In [1]:
!pip install transformers seqeval[gpu]

     |████████████████████████████████| 4.0 MB 11.0 MB/s 
     |████████████████████████████████| 43 kB 1.8 MB/s 
     |████████████████████████████████| 77 kB 5.3 MB/s 
     |████████████████████████████████| 596 kB 32.8 MB/s 
     |████████████████████████████████| 895 kB 31.1 MB/s 
     |████████████████████████████████| 6.6 MB 25.5 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=4ef200b8ddddfa5b42ad721a067ed214599aff5efb30409e7020d2ef45ce0b75
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


All the required libraries

In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizerFast, BertConfig, BertForTokenClassification, AutoTokenizer

In [3]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


Restricting the length of senetces in traning dataset to 50


In [33]:
sentence = []
labels = []
un_l = []
with open('train.txt') as f:
#for BC2GM dataset uncomment the below line and comment the above line
# with open('BC_train.tsv') as f:
    lines = f.readlines()
    hello = 0
    # print(lines.split('\t')) 
    count = 0
    l = ""
    s = ""
    for val in lines:
      k = val.rstrip().split("\t")
      if len(s.split(" "))  == 50:
        #print(len(s.split(" ")))
        sentence.append(s)
        labels.append(l)
        l = ""
        s = ""
      #print(k)
      if len(k) >= 2:
        l = l + k[1] + ","
        un_l.append(k[1])
        s = s + k[0] + " "
      if len(k) < 2 and len(l) > 0:
        sentence.append(s)
        labels.append(l)
        l = ""
        s = ""
    f.close()

tags_ = un_l
ta = {}
un_lab = set(un_l)
unique_labels = []
for i in un_lab:
  unique_labels.append(i)
print(unique_labels)

for i in tags_:
  if i in ta:
    ta[i] += 1
  else:
    ta[i] = 0
print(ta)

data = {'sentence': sentence, 'labels': labels}  
df = pd.DataFrame(data) 
len(df)

df = df.loc[0:, ['sentence', 'labels']]

['B-Target', 'B-Mineral', 'B-Element', 'I-Mineral', 'O']
{'O': 96135, 'B-Target': 712, 'B-Element': 1096, 'B-Mineral': 630, 'I-Mineral': 10}


In [34]:
print(df)

                                               sentence  \
0     . CENTIMETER TO DECIMETER SIZE SPHERICAL AND C...   
1     A.G. FairÃ©n8 , D. Oehler9 , O. Forni2 , V. Sa...   
2     , 7Brown , 8CAB , 9Jacobs LZ , 10MNHN , 11JPL/...   
3     observation has been southward-dipping sedimen...   
4     not been seen by previous landed missions . Th...   
...                                                 ...   
2034  complex also found wide variety of clays , car...   
2035  . Finally , there has been some evidence of re...   
2036  analysis of planetary analogs using multiple t...   
2037  potential processes that have taken place . If...   
2038  : -LSB- 1 -RSB- Grotzinger J. P. et al. -LRB- ...   

                                                 labels  
0     O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,...  
1     O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,...  
2     O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,...  
3     O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,...  
4

Restricting the length of senetces in test dataset to 100


In [35]:
sentence = []
labels = []
un_l = []
with open('train.txt') as f:
#for BC2GM dataset uncomment the below line and comment the above line
# with open('BC_test.txt') as f:
    lines = f.readlines()
    hello = 0
    # print(lines.split('\t')) 
    count = 0
    l = ""
    s = ""
    for val in lines:
      k = val.rstrip().split("\t")
      if len(s.split(" "))  == 100:
        #print(len(s.split(" ")))
        sentence.append(s)
        labels.append(l)
        l = ""
        s = ""
      #print(k)
      if len(k) >= 2:
        l = l + k[1] + ","
        un_l.append(k[1])
        s = s + k[0] + " "
      if len(k) < 2 and len(l) > 0:
        sentence.append(s)
        labels.append(l)
        l = ""
        s = ""
    f.close()

tags_ = un_l
ta = {}
un_lab = set(un_l)
unique_labels = []
for i in un_lab:
  unique_labels.append(i)
print(unique_labels)

for i in tags_:
  if i in ta:
    ta[i] += 1
  else:
    ta[i] = 0
print(ta)

validation_dataset = {'sentence': sentence, 'labels': labels}  
data_ = pd.DataFrame(validation_dataset) 
df_vali = data_.loc[0:, ['sentence', 'labels']]

['B-Target', 'B-Mineral', 'B-Element', 'I-Mineral', 'O']
{'O': 96135, 'B-Target': 712, 'B-Element': 1096, 'B-Mineral': 630, 'I-Mineral': 10}


In [36]:
df_vali

,sentence,labels
0,. CENTIMETER TO DECIMETER SIZE SPHERICAL AND C...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
1,"7Brown , 8CAB , 9Jacobs LZ , 10MNHN , 11JPL/Ca...","O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
2,seen by previous landed missions . The first o...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
3,area . The visible portion is 16 cm in diamete...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
4,. Winnipesaukee : Another feature -LRB- Fig. 2...,"O,B-Target,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O..."
...,...,...
1023,used prototypes with the objective of be used ...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
1024,In situ analysis of different materials were p...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
1025,"of clays , carbonates with varying degrees of ...","O,O,O,B-Mineral,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
1026,techniques thanks to its unique geology and it...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."


converting tags to dictionary

In [39]:
data = df
labels_to_ids = {k: v for v, k in enumerate(unique_labels)}
ids_to_labels = {v: k for v, k in enumerate(unique_labels)}
print(labels_to_ids)
print(ids_to_labels)

{'B-Target': 0, 'B-Mineral': 1, 'B-Element': 2, 'I-Mineral': 3, 'O': 4}
{0: 'B-Target', 1: 'B-Mineral', 2: 'B-Element', 3: 'I-Mineral', 4: 'O'}


In [40]:
MAX_LEN = 512
TRAIN_BATCH_SIZE = 6
VALID_BATCH_SIZE = 6
EPOCHS = 10
#for BC2GM dataset uncomment the below line and comment the above line
# EPOCHS = 5
LEARNING_RATE = 2e-05
#for BC2GM dataset uncomment the below line and comment the above line
# LEARNING_RATE = 2e-03
MAX_GRAD_NORM = 10
MODEL = 'bert-base-uncased'
tokenizer = BertTokenizerFast.from_pretrained(MODEL)

Training the model on the tag labels for the first word piece token of a word.

In [41]:
class dataset(Dataset):
  def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

  def __getitem__(self, index):
        sentence = self.data.sentence[index]
        word_labels = self.data.labels[index].split(",") 
        #The tokenizer to encode sentence which includes padding/truncation up to max length
        #BertTokenizerFast provides "return_offsets_mapping" functionality for individual token
        encoding = self.tokenizer(sentence, return_offsets_mapping=True, padding='max_length', truncation=True, max_length=self.max_len)
        labels = []
        #if the label is empty, just ignore it
        for label in  word_labels:
          if label == " " or label == "":
            pass
          else:
            labels.append(labels_to_ids[label])
        
        #creates token labels only for first word pieces of each tokenized word
        encoded_labels = np.ones(len(encoding["offset_mapping"]), dtype=int) * -100

        # set only labels whose first offset position is 0 and the second is not 0
        i = 0
        for idx, mapping in enumerate(encoding["offset_mapping"]):
          if mapping[0] == 0 and mapping[1] != 0:
            encoded_labels[idx] = labels[i]
            i += 1

        #converting to pytorch tensor
        item = {key: torch.as_tensor(val) for key, val in encoding.items()}
        item['labels'] = torch.as_tensor(encoded_labels)
        
        return item

  def __len__(self):
        return self.len

In [42]:
train_size = 0.80
train_dataset = data.sample(frac=train_size,random_state=200)
test_dataset = data.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)
vali_ = data_.sample(frac=0.999,random_state=200)
validation_dataset  = vali_.reset_index(drop=True)


training_set = dataset(train_dataset, tokenizer, MAX_LEN)
testing_set = dataset(test_dataset, tokenizer, MAX_LEN)
validation_set = dataset(validation_dataset, tokenizer, MAX_LEN)
#testing_set = dataset(train_dataset, tokenizer, MAX_LEN)

In [43]:
print(validation_set)

In [44]:
train_params = {'batch_size': TRAIN_BATCH_SIZE, 'shuffle': True, 'num_workers': 0}
test_params = {'batch_size': VALID_BATCH_SIZE,'shuffle': True,'num_workers': 0 }
vali_params = {'batch_size': VALID_BATCH_SIZE,'shuffle': True,'num_workers': 0 }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)
vali_loader = DataLoader(validation_set, **vali_params)

In [64]:
model = BertForTokenClassification.from_pretrained('bert-base-uncased', num_labels=len(labels_to_ids), output_attentions = False, 
    output_hidden_states = False, 
    #setting dropour probablities to 20%
    hidden_dropout_prob = 0.2,
    attention_probs_dropout_prob = 0.2 )
model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTokenClassification: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-u

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.2, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.2, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [65]:
inputs = training_set[2]
input_ids = inputs["input_ids"].unsqueeze(0)
attention_mask = inputs["attention_mask"].unsqueeze(0)
labels = inputs["labels"].unsqueeze(0)

input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)
labels = labels.to(device)

outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
initial_loss = outputs[0]
initial_loss

tensor(1.4384, device='cuda:0', grad_fn=<NllLossBackward0>)

In [66]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)

In [67]:
train_accuracy = []
train_loss = []

In [69]:
#training the model on 80:20 split
def train(epoch):
    tr_loss, tr_accuracy = 0, 0
    tr_examples, tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    model.train()
    
    for idx, batch in enumerate(training_loader):
        # print(idx)
        ids = batch['input_ids'].to(device, dtype = torch.long)
        mask = batch['attention_mask'].to(device, dtype = torch.long)
        labels = batch['labels'].to(device, dtype = torch.long)

        loss, tr_logits = model(input_ids=ids, attention_mask=mask, labels=labels, return_dict=False)
        tr_loss += loss.item()

        tr_steps += 1
        tr_examples += labels.size(0)
        
        if idx % 200==0:
            loss_step = tr_loss/tr_steps
            print("Training loss per 200 training steps: ",loss_step)
           
        flattened_targets = labels.view(-1) 
        active_logits = tr_logits.view(-1, model.num_labels) 
        flattened_predictions = torch.argmax(active_logits, axis=1) 
        
        #computing accuracy for active labels
        active_accuracy = labels.view(-1) != -100 
        
        labels = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)
        
        tr_labels.extend(labels)
        tr_preds.extend(predictions)

        tmp_tr_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy
    
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=MAX_GRAD_NORM
        )

        #backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    epoch_loss = tr_loss / tr_steps
    tr_accuracy = tr_accuracy / tr_steps
    print("Training loss epoch: ", epoch_loss)
    print("Training accuracy epoch: ", tr_accuracy)
    train_accuracy.append(epoch_loss)
    train_loss.append(tr_accuracy)

In [70]:
for epoch in range(EPOCHS):
    print("Training epoch: ", epoch + 1)
    train(epoch)

Training epoch:  1
Training loss per 200 training steps:  1.8072136640548706
Training loss per 200 training steps:  0.1821453554778179
Training loss epoch:  0.1603578502750572
Training accuracy epoch:  0.9693627450980395
Training epoch:  2
Training loss per 200 training steps:  0.548542857170105
Training loss per 200 training steps:  0.07241469542889749
Training loss epoch:  0.06856729178252863
Training accuracy epoch:  0.9797794117647058
Training epoch:  3
Training loss per 200 training steps:  0.03720363602042198
Training loss per 200 training steps:  0.04158803177140283
Training loss epoch:  0.03431739562887849
Training accuracy epoch:  0.9901960784313727
Training epoch:  4
Training loss per 200 training steps:  0.0014214032562449574
Training loss per 200 training steps:  0.01547759999631121
Training loss epoch:  0.02291964105418575
Training accuracy epoch:  0.996936274509804
Training epoch:  5
Training loss per 200 training steps:  0.001651829108595848
Training loss per 200 trainin

Evaluating the reults of 80:20 split

In [71]:
def valid(model, testing_loader):
  #eval mode
    model.eval()
    
    eval_loss, eval_accuracy = 0, 0
    eval_examples, eval_steps = 0, 0
    eval_preds, eval_labels = [], []
    
    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):
            
            ids = batch['input_ids'].to(device, dtype = torch.long)
            mask = batch['attention_mask'].to(device, dtype = torch.long)
            labels = batch['labels'].to(device, dtype = torch.long)
            
            loss, eval_logits = model(input_ids=ids, attention_mask=mask, labels=labels, return_dict=False)
            
            eval_loss += loss.item()

            eval_steps += 1
            eval_examples += labels.size(0)
        
            if idx % 200==0:
                loss_step = eval_loss/eval_steps
                print("Validation loss per 200 evaluation steps: ",loss_step)
              

            flattened_targets = labels.view(-1) 
            active_logits = eval_logits.view(-1, model.num_labels) 
            flattened_predictions = torch.argmax(active_logits, axis=1) 
            
            #active accuracy
            active_accuracy = labels.view(-1) != -100 
        
            labels = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)
            
            #evaluating
            eval_labels.extend(labels)
            eval_preds.extend(predictions)
            
            tmp_eval_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy

    labels = [ids_to_labels[id.item()] for id in eval_labels]
    predictions = [ids_to_labels[id.item()] for id in eval_preds]
    
    eval_loss = eval_loss / eval_steps
    eval_accuracy = eval_accuracy / eval_steps
    print("Validation Loss: ",eval_loss)
    print("Validation Accuracy: ",eval_accuracy)

    return labels, predictions

In [72]:
labels, predictions = valid(model, testing_loader)

Validation loss per 200 evaluation steps:  1.5706993341445923
Validation Loss:  0.16580272300277155
Validation Accuracy:  0.9730392156862748


In [73]:
from seqeval.metrics import classification_report
from sklearn.metrics import confusion_matrix
# print(confusion_matrix(labels, predictions))
labels = [labels]
predictions = [predictions]
#classification report
print(classification_report(labels, predictions))

              precision    recall  f1-score   support

     Element       0.75      0.67      0.71         9
     Mineral       0.33      0.25      0.29         4
      Target       1.00      0.67      0.80         3

   micro avg       0.69      0.56      0.62        16
   macro avg       0.69      0.53      0.60        16
weighted avg       0.69      0.56      0.62        16



In [74]:
O_ = labels[0].count('O')
B_M = labels[0].count("B-Mineral")
I_M = labels[0].count("I-Mineral")
B_T = labels[0].count("B-Target")
B_E = labels[0].count("B-Element")

print(O_, B_M , I_M, B_T, B_E)

392 4 0 3 9


In [ ]:
# O_ = labels[0].count('O')
# B_g = labels[0].count("B-GENE")
# I_g = labels[0].count("I-GENE")

# print(O_, B_g , I_g)

In [75]:
print(labels)
print(predictions)

[['O', 'O', 'O', 'O', 'O', 'B-Element', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Target', 'B-Element', 'O', 'O', 'O', 'B-Target', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Element', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Mineral', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Element', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O',

Validation on the test dataset

In [76]:
def valid(model, testing_loader):
    model.eval()
    
    eval_loss, eval_accuracy = 0, 0
    eval_examples, eval_steps = 0, 0
    eval_preds, eval_labels = [], []
    
    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):
            
            ids = batch['input_ids'].to(device, dtype = torch.long)
            mask = batch['attention_mask'].to(device, dtype = torch.long)
            labels = batch['labels'].to(device, dtype = torch.long)
            
            loss, eval_logits = model(input_ids=ids, attention_mask=mask, labels=labels, return_dict=False)
            
            eval_loss += loss.item()

            eval_steps += 1
            eval_examples += labels.size(0)
        
            if idx % 200==0:
                loss_step = eval_loss/eval_steps
                print("Validation loss per 200 evaluation steps: ",loss_step)
              

            flattened_targets = labels.view(-1) 
            active_logits = eval_logits.view(-1, model.num_labels) 
            flattened_predictions = torch.argmax(active_logits, axis=1) 
            
            
            active_accuracy = labels.view(-1) != -100 
        
            labels = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)
            
            eval_labels.extend(labels)
            eval_preds.extend(predictions)
            
            tmp_eval_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy

    labels = [ids_to_labels[id.item()] for id in eval_labels]
    predictions = [ids_to_labels[id.item()] for id in eval_preds]
    
    eval_loss = eval_loss / eval_steps
    eval_accuracy = eval_accuracy / eval_steps
    print("Validation Loss: ",eval_loss)
    print("Validation Accuracy: ",eval_accuracy)

    return labels, predictions

In [77]:
labels, predictions = valid(model, vali_loader)

Validation loss per 200 evaluation steps:  7.650954648852348e-05
Validation Loss:  0.0911976559395424
Validation Accuracy:  0.9864341085271321


In [78]:
labels = [labels]
predictions = [predictions]

print(classification_report(labels, predictions))

              precision    recall  f1-score   support

     Element       0.69      0.90      0.78        10
     Mineral       1.00      0.57      0.73         7
      Target       0.60      0.38      0.46         8

   micro avg       0.73      0.64      0.68        25
   macro avg       0.76      0.62      0.66        25
weighted avg       0.75      0.64      0.66        25



In [79]:
O_ = labels[0].count('O')
B_M = labels[0].count("B-Mineral")
I_M = labels[0].count("I-Mineral")
B_T = labels[0].count("B-Target")
B_E = labels[0].count("B-Element")

print(O_, B_M , I_M, B_T, B_E)

1002 7 0 8 10


In [80]:
# O_ = labels[0].count('O')
# B_g = labels[0].count("B-GENE")
# I_g = labels[0].count("I-GENE")

# print(O_, B_g , I_g)

In [81]:
print(labels)

[['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Element', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Mineral', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O

In [82]:
print(predictions)

[['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Element', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Mineral', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O

In [85]:
lables = labels[0]
predictions = predictions[0]

In [86]:
print(lables)
print(predictions)

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Element', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Mineral', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'

In [87]:
print(len(labels[0]))
print(len(predictions))

1027
1027
